In [1]:
import numpy as np
import pandas as pd

import dask.dataframe as dd

from dask.diagnostics import ProgressBar
from dask.distributed import Client
from nemosis import data_fetch_methods, defaults

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
raw_data_cache_location = "./NEM-data"

In [3]:
gen_connection  = data_fetch_methods.static_table_xl('Generators and Scheduled Loads',
                                                     raw_data_location=raw_data_cache_location)
gen_connection

Retrieving static table Generators and Scheduled Loads.


,Participant,Station Name,Region,Dispatch Type,Category,Classification,Fuel Source - Primary,Fuel Source - Descriptor,Technology Type - Primary,Technology Type - Descriptor,Physical Unit No.,Unit Size (MW),Aggregation,DUID,Reg Cap (MW),Max Cap (MW),Max ROC/Min
0,AETV Pty Ltd,Bairnsdale Power Station,VIC1,Generator,Market,Scheduled,Fossil,Natural Gas,Combustion,Open Cycle Gas turbines (OCGT),1,47,Y,BDL01,47,47,4
1,AETV Pty Ltd,Bairnsdale Power Station,VIC1,Generator,Market,Scheduled,Fossil,Natural Gas,Combustion,Open Cycle Gas turbines (OCGT),2,47,Y,BDL02,47,47,4
2,AETV Pty Ltd,Chepstowe Wind Farm,VIC1,Generator,Market,Non-Scheduled,Wind,Wind,Renewable,Wind - Onshore,1-3,2.05,N,CHPSTWF1,6.15,6,0
3,AGL Hydro Partnership,Agl Kemps Creek,NSW1,Generator,Market,Non-Scheduled,Renewable/ Biomass / Waste,Landfill Methane / Landfill Gas,Combustion,Compression Reciprocating Engine,1-2,0.5,Y,AGLSITA1,1,2,0
4,AGL Hydro Partnership,Banimboola Power Station,VIC1,Generator,Market,Non-Scheduled,Hydro,Water,Renewable,Hydro - Gravity,1-2,5.3,Y,BAPS,12.85,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,Wild Cattle Hill Pty Ltd,Cattle Hill Wind Farm,TAS1,Generator,Market,Semi-Scheduled,Wind,Wind,Renewable,Wind - Onshore,1-48,3,Y,CTHLWF1,148,144,6
484,Willogoleche Power Pty Ltd as Trustee for the ...,Willogoleche Wind Farm,SA1,Generator,Market,Semi-Scheduled,Wind,Wind,Renewable,Wind - Onshore,1-24,3.83,Y,WGWF1,91.92,119,50
488,Windy Hill Wind Farm Pty Ltd,Windy Hill Wind Farm,QLD1,Generator,Market,Non-Scheduled,Wind,Wind,Renewable,Wind - Onshore,1-20,0.6,Y,WHILL1,12,12,NaN
489,Woodlawn Wind Pty Ltd,Woodlawn Wind Farm,NSW1,Generator,Market,Semi-Scheduled,Wind,Wind,Renewable,Wind - Onshore,NaN,2.1,N,WOODLWN1,48.3,48,13


In [4]:
fcas_variables = data_fetch_methods.static_table('VARIABLES_FCAS_4_SECOND',
                                                 raw_data_location=raw_data_cache_location)
fcas_variables['VARIABLENUMBER'] = fcas_variables['VARIABLENUMBER'].astype(np.int64)
fcas_variables

Retrieving static table VARIABLES_FCAS_4_SECOND.


,VARIABLENUMBER,VARIABLETYPE
0,1,MW
1,2,Gen_MW
2,3,GenSPD_MW
3,4,GenRPF_%
4,5,GenRegComp_MW
5,6,Spare06
6,7,Spare07
7,8,Spare08
8,9,ON
9,10,OFF


In [5]:
fcas_elements  = data_fetch_methods.static_table('ELEMENTS_FCAS_4_SECOND',
                                                raw_data_location=raw_data_cache_location)
fcas_elements['ELEMENTNUMBER'] = fcas_elements['ELEMENTNUMBER'].astype(np.int64)
fcas_elements

Retrieving static table ELEMENTS_FCAS_4_SECOND.


,ELEMENTNUMBER,EMSNAME,ELEMENTTYPE,MMSDESCRIPTOR
0,1,SUBSTN.LYPA.GEN.A1GEN,GEN,*MMS MarketName*
1,2,SUBSTN.LYPA.GEN.A2GEN,GEN,*MMS MarketName*
2,3,SUBSTN.LYPA.GEN.A3GEN,GEN,*MMS MarketName*
3,4,SUBSTN.LYPA.GEN.A4GEN,GEN,*MMS MarketName*
4,5,SUBSTN.LYPB.GEN.B1GEN,GEN,*MMS MarketName*
...,...,...,...,...
368,20021,SPD VIC-NSW,INTERCONNECTOR,*MarketName*
369,20022,SPD Basslink,INTERCONNECTOR,*MarketName*
370,10001,APD SUMM POTLINES,LOAD,*MarketName*
371,10006,TOMAGO SUMM POTLINES,LOAD,*MarketName*


In [29]:
pbar=ProgressBar()
pbar.register()

client = Client()

/home/abi/.local/share/virtualenvs/nem-data-analysis-PJjZKztB/lib/python3.6/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [30]:
ddf = dd.read_parquet('./NEM-data/FCAS_20200202*.parquet')

In [31]:
ddf['TIMESTAMP'] = ddf['TIMESTAMP'].astype(np.datetime64)
ddf['ELEMENTNUMBER'] = ddf['ELEMENTNUMBER'].astype(int)
ddf['VARIABLENUMBER'] = ddf['VARIABLENUMBER'].astype(int)
ddf['VALUE'] = ddf["VALUE"].astype(np.float64)
ddf['VALUEQUALITY'] = ddf['VALUEQUALITY'].astype(int)

In [33]:
f = freq_data.compute()